In [1]:
import requests
import folium
import pandas as pd
from folium import IFrame

# Fetch the data from the API
url = 'http://127.0.0.1:9090/airquality/copd'
response = requests.get(url)
data = response.json()

# Convert JSON data to a DataFrame
df = pd.json_normalize(data)

# Find the maximum values for total, male, and female COPD admissions
max_total = df['average_total_copd_admissions_num'].max()
max_male = df['average_male_copd_admissions_num'].max()
max_female = df['average_female_copd_admissions_num'].max()

# Create a Folium map centered on a specific location
m = folium.Map(width=900, height=600, location=[-36.5115, 144.1652], zoom_start=7, tiles='OpenStreetMap')

# Define a function to create styled HTML for the popups
def create_popup_html(row):
    html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <h4>{row['site_name']}</h4>
        <p><strong>Health Advice:</strong> <span style="color: {row['health_advice_color']};">{row['health_advice'] if not row['health_advice'] is None else '---'}</span></p>
        <p><strong>Value:</strong> {row['value'] if not pd.isna(row['value']) else '---'} {row['unit'] if not row['unit'] is None else ''}</p>
        <p><strong>Average Chronic Obstructive Pulmonary Disease Admissions:</strong></p>
        <ul>
            <li>Total: {int(row['average_total_copd_admissions_num'])} people</li>
            <li>Male: {int(row['average_male_copd_admissions_num'])} people</li>
            <li>Female: {int(row['average_female_copd_admissions_num'])} people</li>
        </ul>
    </div>
    """
    return html

# Iterate over the DataFrame and add markers to the map
for index, row in df.iterrows():
    popup_html = create_popup_html(row)
    iframe = IFrame(popup_html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=300)
    
    # Determine the icon color based on the maximum values
    icon_color = "blue"
    if row['average_male_copd_admissions_num'] == max_male or row['average_female_copd_admissions_num'] == max_female:
        icon_color = "red"
    
    folium.Marker(
        location=[row['site_location.lat'], row['site_location.lon']],
        popup=popup,
        tooltip=f"<strong>{row['site_name']}</strong>",
        icon=folium.Icon(color=icon_color, icon="info-sign"),
    ).add_to(m)

# Display the map
m
